In [1]:
import re
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.graphs import Neo4jGraph
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.text_splitter import TokenTextSplitter

from secret import *
import chromadb

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="test1234")
graph.query(
    """
MATCH (n)
DETACH DELETE n
"""
)
graph.refresh_schema()
print(graph.schema)


        Node properties are the following:
        []
        Relationship properties are the following:
        []
        The relationships are the following:
        []
        


In [3]:
from loader.radiopaedia import RadiopaediaLoader
radio_raw_documents = RadiopaediaLoader(query="Atelectasis", diffbot_api_token=DIFFBOT_API_TOKEN).load()
wiki_raw_documents = WikipediaLoader(query="Atelectasis").load()
raw_documents = radio_raw_documents + wiki_raw_documents

In [4]:
len(radio_raw_documents), len(wiki_raw_documents)

(20, 10)

In [5]:
text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)
documents = text_splitter.split_documents(
    raw_documents
)  # should I only take the first? which is the most relevant?

In [6]:
db = Neo4jVector.from_documents(
    documents, OpenAIEmbeddings(), url=NOE4J_URL, username="neo4j", password="test1234"
)

retriever = db.as_retriever()
# 4s

In [7]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI

In [8]:
chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"), chain_type="stuff", retriever=retriever
)

In [9]:
chain(
    {"question": "What is Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Atelectasis is the collapse or closure of a lung resulting in reduced or absent gas exchange. It is usually unilateral, affecting part or all of one lung. It is a condition where the alveoli are deflated down to little or no volume. It can be caused by normal exhalation or various medical conditions. Atelectasis is not synonymous with a pneumothorax, although a pneumothorax can cause atelectasis. It can occur as a post-operative complication or as a result of surfactant deficiency. It is a common finding in chest X-rays and other radiological studies. The most common cause is post-surgical atelectasis, which develops in 75-90% of people undergoing general anesthesia for a surgical procedure. Other causes include pulmonary tuberculosis, obstruction of the airway, poor surfactant spreading, and compression of the lung by a space-occupying lesion. Clinically significant atelectasis is generally visible on chest X-ray and may require further imaging or bronchoscopy for diagnosi

In [10]:
chain(
    {"question": "What are the symptoms of Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'The symptoms of Atelectasis may include cough (not prominent), chest pain (not common), breathing difficulty (fast and shallow), low oxygen saturation, pleural effusion (transudate type), cyanosis (late sign), and increased heart rate. However, it is important to note that atelectasis can also be asymptomatic. \n',
 'sources': 'https://en.wikipedia.org/wiki/Atelectasis, https://radiopaedia.org/articles/atelectasis-summary?lang=gb, https://radiopaedia.org/articles/passive-atelectasis-1?lang=gb, https://radiopaedia.org/articles/lung-atelectasis?lang=gb'}

In [11]:
chain(
    {"question": "What can cause Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Atelectasis can be caused by various factors, including post-surgical complications, surfactant deficiency, pulmonary tuberculosis, blockage of the airway, compression from external factors, poor surfactant spreading, and certain risk factors such as type of surgery, obesity, high oxygen, and presence of chronic obstructive pulmonary disease or asthma. It can also occur due to passive atelectasis or adhesive atelectasis. \n',
 'sources': ''}

In [12]:
chain(
    {"question": "What clinical features radiologists need for diagnosing Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Radiologists need to look for small volume linear shadows on chest x-rays, usually peripherally or at the lung bases, to diagnose atelectasis. They can also use CT scans to accurately depict the involved lung and determine the cause. Additionally, the presence of the crow feet sign, which appears as linear bands radiating from a mass into adjacent lung tissue, can be a characteristic feature of round atelectasis. The clinical presentation of atelectasis can vary depending on the extent and speed of development, but common symptoms include breathlessness, cough, and diminished breath sounds. The underlying causes of atelectasis can include adjacent compression, passive atelectasis, and dependent atelectasis. Treatment involves managing the underlying cause and promoting deeper breathing if hypoventilation is the cause. \n',
 'sources': ''}

In [13]:
chain(
    {"question": "what are radiographic features of Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Radiographic features of atelectasis include small volume linear shadows on chest x-rays, usually peripherally or at the lung bases. The underlying cause may also be visible. CT chest allows for a more accurate depiction of the involved lung and can help determine the cause. Round atelectasis can be seen on CT as linear bands radiating from a mass into adjacent lung tissue, resembling the feet of a crow. Atelectasis can also be classified based on underlying mechanism (resorptive, passive, compressive, cicatrisation, adhesive, gravity dependent) or morphology (linear, round, lung, lobar, segmental, subsegmental). The radiographic features vary depending on the type and mechanism of atelectasis. \n',
 'sources': ''}